#### Simple Gen AI APP Using Langchain

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [4]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

In [5]:
loader=WebBaseLoader("https://docs.langchain.com/langsmith/prompt-commit")
loader

In [7]:
# Loads website information into a single document.
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/prompt-commit', 'title': 'How to sync prompts with GitHub - Docs by LangChain', 'language': 'en'}, page_content='How to sync prompts with GitHub - Docs by LangChainSkip to main contentDocs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationTutorialsHow to sync prompts with GitHubGet startedObservabilityEvaluationPrompt engineeringDeploymentPlatform setupReferenceOverviewQuickstartConceptsCreate and update promptsCreate a promptManage promptsManage prompts programmaticallyPrompt template formatConfigure prompt settingsUse tools in a promptInclude multimodal content in a promptWrite your prompt with AIConnect to modelsTutorialsOptimize a classifierSync prompts with GitHubTest multi-turn conversationsOn this pagePrerequisitesUnderstanding LangSmith “Prompt commits” and webhooksImplementing a FastAPI server for webhook receptionConfiguring the webhook in LangSmithThe workflow

In [8]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [ ]:
# Converted to multiple documents of small chunks as declared in text splitter.
documents

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/prompt-commit', 'title': 'How to sync prompts with GitHub - Docs by LangChain', 'language': 'en'}, page_content='How to sync prompts with GitHub - Docs by LangChainSkip to main contentDocs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationTutorialsHow to sync prompts with GitHubGet startedObservabilityEvaluationPrompt engineeringDeploymentPlatform setupReferenceOverviewQuickstartConceptsCreate and update promptsCreate a promptManage promptsManage prompts programmaticallyPrompt template formatConfigure prompt settingsUse tools in a promptInclude multimodal content in a promptWrite your prompt with AIConnect to modelsTutorialsOptimize a classifierSync prompts with GitHubTest multi-turn conversationsOn this pagePrerequisitesUnderstanding LangSmith “Prompt commits” and webhooksImplementing a FastAPI server for webhook receptionConfiguring the webhook in LangSmithThe workflow

In [10]:
#Create a Open AI Embeddings objects
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [11]:
# Store the created documents into vectorstoreDB called FAISS(Facebook Artificial Intelligent Similarity Search)
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [12]:
vectorstoredb

In [13]:
## Query From a vector db
query="If your application loads prompts from a database"
result=vectorstoredb.similarity_search(query)
result[0].page_content

'While you can dynamically fetch prompts from LangSmith into your application at runtime, you may prefer to sync prompts with your own database or version control system. To support this workflow, LangSmith allows you to receive notifications of prompt updates via webhooks.\nWhy sync prompts with GitHub?'

In [14]:
#Create LLM 
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-2025-04-14", 
                 temperature=0,     
                 timeout=20,
                 max_retries=2)

In [15]:
## Retrieval Chain, Document chain

from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(profile={}, client=<openai.resources.chat.completions.completions.Completions object at 0x000002194C367880>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002196ED89AB0>, root_client=<openai.OpenAI object at 0x000002194A0DBD30>, root_async_client=<openai.AsyncOpenAI object at 0x000002196ED8BD90>, model_name='gpt-4.1-2025-04-14', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********

In [ ]:
# Improved similarity search
from langchain_core.documents import Document
document_chain.invoke({
    "input":"If your application loads prompts from a database",
    "context":[Document(page_content="If your application loads prompts from a database or cache, update these stores directly.")]
})

'If your application loads prompts from a database or cache, what should you do to update them?\n\nYou should update these stores directly.'

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [17]:
### Input--->Retriever--->vectorstoredb

vectorstoredb

In [18]:
retriever=vectorstoredb.as_retriever()
from langchain_classic.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [19]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002196ED899F0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
            | 

In [20]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"LIf your application loads prompts from a database"})
response['answer']

'**Question:** Why sync prompts with GitHub?\n\n**Answer (based only on the provided context):**\n\nSyncing prompts with GitHub offers several benefits:\n\n- **Granular Commits:** You can parse the prompt manifest and commit changes to individual prompt files, allowing for a more detailed and organized structure in your repository.\n- **Version Control:** Prompts are versioned alongside your application code in a familiar system, making it easier to track changes and collaborate.\n- **CI/CD Integration:** Syncing with GitHub enables you to trigger automated staging or production deployments (e.g., via GitHub Actions) when critical prompts change.\n- **Trigger CI/CD Pipelines:** You can have the server trigger a CI/CD pipeline to deploy environments, run tests, or build new application versions when prompts are updated.\n- **Update Databases/Caches:** If your application loads prompts from a database or cache, syncing allows you to update these stores directly.\n- **Notifications:** You

In [21]:

response

{'input': 'LIf your application loads prompts from a database',
 'context': [Document(id='462f85dd-52b6-450e-a7c1-35e014edfb15', metadata={'source': 'https://docs.langchain.com/langsmith/prompt-commit', 'title': 'How to sync prompts with GitHub - Docs by LangChain', 'language': 'en'}, page_content='While you can dynamically fetch prompts from LangSmith into your application at runtime, you may prefer to sync prompts with your own database or version control system. To support this workflow, LangSmith allows you to receive notifications of prompt updates via webhooks.\nWhy sync prompts with GitHub?'),
  Document(id='56630ae0-1696-4071-99c3-10528cc8ff40', metadata={'source': 'https://docs.langchain.com/langsmith/prompt-commit', 'title': 'How to sync prompts with GitHub - Docs by LangChain', 'language': 'en'}, page_content='Granular Commits: Parse the manifest and commit changes to individual prompt files if you prefer a more granular structure in your repository.\nTrigger CI/CD: Instead 

In [22]:
response['context']

[Document(id='462f85dd-52b6-450e-a7c1-35e014edfb15', metadata={'source': 'https://docs.langchain.com/langsmith/prompt-commit', 'title': 'How to sync prompts with GitHub - Docs by LangChain', 'language': 'en'}, page_content='While you can dynamically fetch prompts from LangSmith into your application at runtime, you may prefer to sync prompts with your own database or version control system. To support this workflow, LangSmith allows you to receive notifications of prompt updates via webhooks.\nWhy sync prompts with GitHub?'),
 Document(id='56630ae0-1696-4071-99c3-10528cc8ff40', metadata={'source': 'https://docs.langchain.com/langsmith/prompt-commit', 'title': 'How to sync prompts with GitHub - Docs by LangChain', 'language': 'en'}, page_content='Granular Commits: Parse the manifest and commit changes to individual prompt files if you prefer a more granular structure in your repository.\nTrigger CI/CD: Instead of (or in addition to) committing, have the server trigger a CI/CD pipeline (